In [22]:
import numpy as np # arrays & loading data

from sklearn.model_selection import train_test_split # we will split the data in 3 (training + counter variation + testing data)
from sklearn.preprocessing import StandardScaler # z-score normalization & polynomianls classes 

import tensorflow as tf # for building and training neural networks

from utils import build_models 

In [23]:
# loading all the data
data = np.loadtxt('./food_data.csv', delimiter=',', skiprows=1, usecols=range(1, 7))

X = data[:,:-1] # forming the input and output 
y= data[:,-1]

y= np.expand_dims(y, axis=1) # make y 2D - the commands later will require it

print(f"shape of input X is: {X.shape}")
print(f"shape of output y is: {y.shape}")

shape of input X is: (25, 5)
shape of output y is: (25, 1)


In [24]:
# split into training , cross validation and test sets

# TRAINING SET - 60%
X_train, X_temporary, y_train, y_temporary = train_test_split(X, y, test_size=0.40, random_state=1)

# the rest of 40% - CV SET(20%) and TEST SET(20%)
X_cv, X_test, y_cv, y_test = train_test_split(X_temporary, y_temporary, test_size=0.50, random_state=1)
del X_temporary, y_temporary

print(f"training input shape:{X_train.shape}")
print(f"training output shape:{y_train.shape}")
print(f"cv input shape:{X_cv.shape}")
print(f"cv output shape:{y_cv.shape}")
print(f"test input shape:{X_test.shape}")
print(f"test output shape:{y_test.shape}")

training input shape:(15, 5)
training output shape:(15, 1)
cv input shape:(5, 5)
cv output shape:(5, 1)
test input shape:(5, 5)
test output shape:(5, 1)


In [25]:
# TRAINING DATA, CV DATA & TEST DATA OPERATIONS
# applying z-score to all the training data - make it compact for the algorithms to work better
standard_scaler = StandardScaler()
X_train_scaled = standard_scaler.fit_transform(X_train)
X_cv_scaled = standard_scaler.transform(X_cv) 
X_test_scaled = standard_scaler.transform(X_test)
print(X_train_scaled)

[[ 8.40496830e-02  1.98211614e-01 -6.69481473e-01 -5.46857927e-01
   3.04717808e+00]
 [-6.28410775e-01 -5.45081938e-01 -4.65652391e-01 -2.22223254e-01
  -5.85199668e-01]
 [-6.07759747e-01 -5.28227209e-01 -3.60098045e-01 -2.26551717e-01
  -5.09839134e-01]
 [ 9.10090793e-01 -1.57423169e-01  6.95445416e-01 -5.59843314e-01
  -1.48108570e-01]
 [-6.07759747e-01 -5.45081938e-01 -5.05690246e-01 -3.65062510e-01
  -5.55055454e-01]
 [-2.25715733e-01 -5.11372480e-01 -6.14884397e-01 -5.33872540e-01
   2.61249852e-03]
 [-5.87108719e-01 -5.28227209e-01 -6.18524202e-01 -3.60734048e-01
  -5.85199668e-01]
 [ 1.63287677e+00  2.05054634e+00 -7.71396014e-01 -5.42529465e-01
   1.60025582e+00]
 [-5.35481150e-01 -4.43953563e-01 -6.14884397e-01 -3.56405586e-01
  -1.48108570e-01]
 [-1.22460595e-01  2.47090328e-01  2.84293039e+00  3.19293350e+00
  -6.30415988e-01]
 [ 2.89258946e+00  2.82586387e+00  1.37244915e+00  1.65632938e+00
  -7.27480356e-02]
 [-2.99026882e-01 -4.27098834e-01  1.04486670e+00 -3.86271976e-01

In [27]:
# calculate errors for some models and chose the one with the best neural network
train_errors=[]
cv_errors=[]    # initialize lists that will contain a history of error numbers at each iteration

# this is the list of all the models we will check
models = build_models()

#loop through all the models
for model in models:
    # compile - set the loss and optimizer
    model.compile(
        # this is similar to gradient descent, but it is a much improved version
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), # BC - binary class & from_logits helps our calculations be more accurate
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), # preimplemented optimizer
    )
    
    # train the model "epochs" times
    model.fit(
        X_train_scaled, y_train,
        epochs=200,
        verbose=0
    )
    # threshold 0.5 because we are in a binary class example

    # fraction of misclassified outputs in the TRAINING SET
    y_predicted = model.predict(X_train_scaled)
    y_predicted = tf.math.sigmoid(y_predicted) # classify above or below the threshold
    # classic way of calculating the error in a classification case
    y_predicted = np.where(y_predicted>=0.5,1,0)
    error = np.mean(y_predicted != y_train) # arithmetic mean, where the numerator is the count of wrong predictions
    train_errors.append(error)
    
    # fraction of misclassified outputs in the CV SET
    y_predicted = model.predict(X_cv_scaled)
    y_predicted = tf.math.sigmoid(y_predicted) 
    y_predicted = np.where(y_predicted>=0.5,1,0)
    error2 = np.mean(y_predicted != y_cv) # arithmetic mean, where the numerator is the count of wrong predictions
    cv_errors.append(error2)
    
# Print the result
for model_num in range(len(train_errors)):
    print(
        f"Model {model_num+1}: Training Set Classification Error: {train_errors[model_num]:.5f}, " +
        f"CV Set Classification Error: {cv_errors[model_num]:.5f}"
        )

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Model 1: Training Set Classification Error: 0.00000, CV Set Classification Error: 0.20000
Model 2: Training Set Classification Error: 0.00000, CV Set Classification Error: 0.20000
Model 3: Training Set Classification Error: 0.00000, CV Set Classification Error: 0.20000
Model 4: Training Set Classification Error: 0.00000, CV Set Classification Error: 0.20000
Model 5: Training Set Classification Error: 0.00000, CV Set Classification Error: 0.20000
